In [ ]:
import requests
import json
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def get_keystr(driver):
    driver.get(url)
    driver.refresh()
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it("iframeResult"))
    search_button_locator = (By.XPATH, '//input[@type="submit" and @value="地址定位"]')
    search_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable(search_button_locator))
    search_button.click()
    time.sleep(5)
    networkScript = """
    var performance = window.performance || window.webkitPerformance || {};
    var network = performance.getEntries() || {};
    return network;
    """
    networkRequests = driver.execute_script(networkScript)

    # Filter URLs that start with "TGA" (or "TGL", depending on what you're targeting)
    for request in networkRequests:
        URLs  = request.get('name', '')
        if 'TGA' in URLs  and 'keystr=' in URLs:
            # Extract keystr value from URL query parameters
            from urllib.parse import urlparse, parse_qs
            parsed_url = urlparse(URLs )
            query_params = parse_qs(parsed_url.query)
            keystr_list = query_params.get('keystr', [])
            if keystr_list:
                keystr = keystr_list[0]
                return keystr
cookies = {
    '_ga': 'GA1.2.2139804078.1676556517',
    '_gid': 'GA1.2.651109627.1679290300',
    '_ga_14NXYHNJ60': 'GS1.1.1679290300.4.0.1679290303.0.0.0',
    'GCLB': '"ad54252a126e9a14"',
    'ai_user': 'N8+c8Em8RvLvnVS2R58OmH|2023-03-20T05:38:00.207Z',
    'GCILB': '"592aff647ed9f325"',
}

headers = {
    'authority': 'gis.tgos.tw',
    'accept': '*/*',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'cookie': '_ga=GA1.2.2139804078.1676556517; _gid=GA1.2.651109627.1679290300; _ga_14NXYHNJ60=GS1.1.1679290300.4.0.1679290303.0.0.0; GCLB="ad54252a126e9a14"; ai_user=N8+c8Em8RvLvnVS2R58OmH|2023-03-20T05:38:00.207Z; GCILB="592aff647ed9f325"',
    'referer': 'https://map.tgos.tw/',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
}
url = 'https://api.tgos.tw/TGOS_MAP_API/Docs/Example/99'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--auto-open-devtools-for-tabs")
driver = webdriver.Chrome(options=chrome_options)

try:
    ################################## file path ##########################
    input_path = r'C:\Users\USER\Desktop\test.csv'
    output_path = r'C:\Users\USER\Desktop\Addresss_result.csv'
    #######################################################################
    start_time = time.time() 
    keystr = get_keystr(driver)
    print(keystr)

    df = pd.read_csv(input_path, encoding="utf-8")
    ################## column name for the address of the csv file ##################
    df1 = df['商業地址']
    #################################################################################

    arr = df1.to_numpy()

    with open(output_path, 'a', newline='', encoding="utf_8_sig") as csvfile:
        writer = csv.writer(csvfile)

        header = ['地址', 'X', 'Y']
        if csvfile.tell() == 0:
            writer.writerow(header)

        for i in arr:
            response = requests.post(
                'https://gis.tgos.tw/TGAddress/TGAddress.aspx?oAddress='+i+'&oSRS=EPSG:3826&oResultDataType=jsonp&pnum=NaN&keystr='+keystr+'&jsonp=TGOS.getJSON%5B%27sn2%27%5D',    
                cookies=cookies,
                headers=headers,
            )
            result = response.content
            result = result.decode()
            result = result.replace("TGOS.getJSON['sn2'](", "")
            result = result.rstrip(");")
            try:
                data = json.loads(result)
                if 'AddressList' in data and data['AddressList']:
                    result = data['AddressList'][0]

                    if 'FULL_ADDR' in result and 'X' in result and 'Y' in result:
                        adr = result['FULL_ADDR']
                        x = result['X']
                        y = result['Y']

                        print(adr, x, y)

                else:
                    adr = "-9999999"
                    x = "-9999999"
                    y = "-9999999"

                    print(adr, x, y)

                writer.writerow([adr, x, y])

            except IndexError:
                pass

            elapsed_time = time.time() - start_time  
            if elapsed_time >= 1000:
                keystr = get_keystr(driver)
                print(keystr)
                start_time = time.time()  


except TypeError as te:
    if "can only concatenate str (not 'float') to str" in str(te):
        pass  

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    driver.quit()


In [ ]:
import requests
import json
import csv
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs

TGOS_URL = 'https://api.tgos.tw/TGOS_MAP_API/Docs/Example/99'

def get_keystr():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get(TGOS_URL)
        driver.refresh()
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it("iframeResult"))
        search_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//input[@type="submit" and @value="地址定位"]'))
        )
        search_button.click()
        time.sleep(5)
        network_requests = driver.execute_script("""
            var performance = window.performance || {};
            return performance.getEntries() || [];
        """)
        for request in network_requests:
            url_str = request.get('name', '')
            if 'TGA' in url_str and 'keystr=' in url_str:
                query_params = parse_qs(urlparse(url_str).query)
                keystr_list = query_params.get('keystr', [])
                if keystr_list:
                    return keystr_list[0]
    finally:
        driver.quit()
    return None

def query_address(address, keystr, session):
    try:
        params = {
            'oAddress': address,
            'oSRS': 'EPSG:3826',
            'oResultDataType': 'jsonp',
            'pnum': 'NaN',
            'keystr': keystr,
            'jsonp': "TGOS.getJSON['sn2']"
        }
        response = session.post("https://gis.tgos.tw/TGAddress/TGAddress.aspx", params=params)
        result = response.text.replace("TGOS.getJSON['sn2'](", "").rstrip(");")
        data = json.loads(result)
        if 'AddressList' in data and data['AddressList']:
            info = data['AddressList'][0]
            return [address, info.get('X', '-9999999'), info.get('Y', '-9999999')]
    except:
        pass
    return [address, '-9999999', '-9999999']

def main():
    ################################## file path ##########################
    input_path = r'C:\Users\USER\Desktop\test.csv'
    output_path = r'C:\Users\USER\Desktop\Addresss_result.csv'
    ########################################################################
    df = pd.read_csv(input_path, encoding="utf-8")
    ################## Name of the address column in the CSV file ##################
    address_list = df['商業地址'].dropna().tolist()
    #################################################################################
    keystr = get_keystr()
    last_keystr_time = time.monotonic()
    print(f"[INFO] Initial keystr obtained: {keystr}")

    headers = {
        'user-agent': 'Mozilla/5.0',
        'referer': 'https://map.tgos.tw/',
    }
    cookies = {
        '_ga': 'GA1.2.2139804078.1676556517',
        '_gid': 'GA1.2.651109627.1679290300',
    }

    results = []

    with requests.Session() as session:
        session.headers.update(headers)
        session.cookies.update(cookies)

        def safe_query(addr):
            nonlocal keystr, last_keystr_time
            if time.monotonic() - last_keystr_time > 1000:
                print("[INFO] Refreshing keystr...")
                new_keystr = get_keystr()
                if new_keystr:
                    keystr = new_keystr
                    last_keystr_time = time.monotonic()
                    print(f"[INFO] New keystr obtained: {keystr}")
            return query_address(addr, keystr, session)

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {
                executor.submit(safe_query, addr): addr
                for addr in address_list
            }
            for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
                try:
                    results.append(future.result())
                except Exception as e:
                    print(f"[ERROR] {e}")

    with open(output_path, 'w', newline='', encoding="utf_8_sig") as f:
        writer = csv.writer(f)
        writer.writerow(['Address', 'X', 'Y'])
        writer.writerows(results)

    print("✅ Done!")

if __name__ == "__main__":
    main()
